In [1]:
import torch
import numpy as np
import scipy.sparse as sp
import gc

from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN
from deeprobust.graph.global_attack import Metattack
from deeprobust.graph.utils import normalize_adj, sparse_mx_to_torch_sparse_tensor

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# 1. Load the Cora dataset
data = Dataset(root='/tmp/', name='cora', setting='nettack')
adj, features, labels = data.adj, data.features, data.labels
idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
idx_unlabeled = np.union1d(idx_val, idx_test)

# 2. Train GCN on the clean graph
model_clean = GCN(nfeat=features.shape[1],
                  nclass=labels.max().item() + 1,
                  nhid=64,
                  with_relu=False,
                  dropout=0.5,
                  device=device)
model_clean = model_clean.to(device)
model_clean.fit(features, adj, labels, idx_train)

# 3. Ensure features and labels are PyTorch tensors on GPU for inference
if isinstance(features, np.ndarray):
    features = torch.FloatTensor(features).to(device)
elif not isinstance(features, torch.Tensor):
    features = torch.FloatTensor(features.A).to(device)
else:
    features = features.to(device)

if isinstance(labels, np.ndarray):
    labels = torch.LongTensor(labels).to(device)

# 4. Evaluate accuracy on the clean graph
adj_norm = normalize_adj(adj)
adj_tensor = sparse_mx_to_torch_sparse_tensor(adj_norm).to(device)

model_clean.eval()
output_clean = model_clean.predict(features, adj_tensor)
preds_clean = output_clean.argmax(dim=1)

correct_clean = preds_clean[idx_test].eq(labels[idx_test]).sum().item()
acc_clean = correct_clean / len(idx_test)
print(f"✅ Clean Graph Test Accuracy: {acc_clean:.4f}")

gc.collect()

# 5. Prepare for Metattack (move inputs to CPU)
features_cpu = features.cpu()
labels_cpu = labels.cpu()

gc.collect()

# 6. Apply evasion attack (perturb structure after training)
attack_model = Metattack(model=model_clean,
                         nnodes=adj.shape[0],
                         feature_shape=features.shape,
                         attack_structure=True,
                         attack_features=False,
                         device=device)
attack_model = attack_model.to(device)

perturbations = int(0.20 * (adj.sum() // 2))  # 5% of total edges
attack_model.attack(features_cpu, adj, labels_cpu, idx_train, idx_unlabeled, perturbations, ll_constraint=False)
modified_adj = attack_model.modified_adj

gc.collect()

# 7. Normalize and convert modified adj to PyTorch sparse tensor
if isinstance(modified_adj, torch.Tensor):
    modified_adj = modified_adj.cpu().numpy()
    modified_adj = sp.csr_matrix(modified_adj)

modified_adj_norm = normalize_adj(modified_adj)
modified_adj_tensor = sparse_mx_to_torch_sparse_tensor(modified_adj_norm).to(device)

gc.collect()

# 8. Evaluate model on the attacked graph
output_adv = model_clean.predict(features, modified_adj_tensor)
preds_adv = output_adv.argmax(dim=1)

correct_adv = preds_adv[idx_test].eq(labels[idx_test]).sum().item()
acc_evasion = correct_adv / len(idx_test)
print(f"🧨 Evasion Attack Test Accuracy: {acc_evasion:.4f}")

# 9. Compute and report accuracy drop
acc_drop = acc_clean - acc_evasion
print(f"📉 Accuracy Drop due to Evasion Attack: {acc_drop:.4f}")


Loading cora dataset...
Selecting 1 largest connected components
✅ Clean Graph Test Accuracy: 0.8053


Perturbing graph:   0%|                                                                        | 0/1013 [00:00<?, ?it/s]

GCN loss on unlabled data: 0.5076307654380798
GCN acc on unlabled data: 0.8484577559231113
attack loss: 0.17009223997592926


Perturbing graph:   0%|                                                                | 1/1013 [00:01<17:55,  1.06s/it]

GCN loss on unlabled data: 0.5261403322219849
GCN acc on unlabled data: 0.8489047831917748
attack loss: 0.16797800362110138


...

GCN loss on unlabled data: 2.01560378074646
GCN acc on unlabled data: 0.6571300849351811
attack loss: 1.1579840183258057


Perturbing graph:  41%|█████████████████████████▍                                    | 416/1013 [03:50<05:30,  1.81it/s]

GCN loss on unlabled data: 2.0280888080596924
GCN acc on unlabled data: 0.6566830576665177
attack loss: 1.1657731533050537


Perturbing graph:  41%|█████████████████████████▌                                    | 417/1013 [03:51<05:30,  1.80it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 31.74 GiB total capacity; 31.29 GiB already allocated; 7.12 MiB free; 31.30 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF